In [7]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import pandas as pd


In [3]:
xls = pd.ExcelFile("../megaparse/cdp/exemple_fournisseur.xlsx")
sheets = pd.read_excel(xls, 'Fiche recette')

In [4]:
def table_to_text(df):
    # Initialize an empty list to store the text rows
    text_rows = []

    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # Combine all columns into a single string separated by " | "
        row_text = " | ".join(str(value) for value in row.values if pd.notna(value))

        # Append the combined string to the list
        if row_text:
            text_rows.append("|" + row_text + "|")

    # Join all rows into a single string separated by newlines
    #result_text = "\n".join(text_rows)

    return text_rows

In [5]:
target_text = table_to_text(sheets)
target_text = "\n".join(target_text)


In [6]:
target_text

"|► Recette|\n|Merci de détailler ci-dessous la composition exacte du produit tel qu’il est proposé à ARYZTA. La recette doit être découpée par semi-fini et les % de matières premières exprimés par semi-fini. Les ingrédients des MP composées sont à détailler également. Un exemple de déclaration est proposé dans l'onglet EXEMPLE DECLARATION RECETTE.|\n|Type | Dénomination Semi-Fini | % SF | Dénomination Matière Première | % MP | Détail ingrédients si Matière première composée | Pays d'origine / Zone FAO \n(pour les viandes et les poissons) | Informations supplémentaires |\n|Semi-Fini | 330200 MOUSSE FRAMBOISE MOINS SUCRE | 41.5|\n|Matière première | 1120028 FRAMBOISE PUREE SUCREE 10pourcent | 34.982508745627186 | Serbie-Bosnie&Herzegovine|\n|Matière première | 1050004 EAU | 5.997001499250375 | France|\n|Matière première | 1020001 SUCRE CRISTAL | 5.997001499250375 | UE|\n|Matière première | 1150038 LAMEQUICK CE RSPO | 1.949025487256372 | sirop de glucose, huile de palmiste, émulsifiant :

In [18]:
# prompt = PromptTemplate(
#     template="""
# You are a question generator to verify a given document is following the requirements to be accepted by the company. You are given an excel table transformed into a markdown text.
# Here is the specific context of the markdown text:
# {specific_context}

# Here is the markdown text:
# {target_text}

# Please generate a question to verify the document is following the requirements to be accepted by the company. The question should be a yes/no question.
# Respond in the same language than the markdown text.
# Questions : """,
#     input_variables=["specific_context","target_text"],
# )

In [ ]:
generic_prompt = """"
You will be generating questions to verify the compliance of elements based on a reference document that is not given and an applying document. 

Here is the specific context of the applying document:
<specific context>
{specific_context}
<specific context>


Here is the applying document text in md:
<applying document>
{target_text}
</applying document>

Please generate a question to verify the document is following the requirements to be accepted by the company. The question should be a yes/no question.
Respond in the same language than the markdown text.
Questions : """

In [98]:
generic_prompt = """You will be generating questions to verify the compliance of items based on a provided document. Here is the document:
<document>
{DOCUMENT}
</document>

<document_context>
{DOCUMENT_CONTEXT}
</document_context>

Please follow these steps to generate the questions:

Carefully review the document and items table to understand the context and requirements. Review the input language to know in which language you should generate the questions.
Extract the individual raw items from the items table. For composed items, break them down into their sub-items. Focus on the raw items only, not the composed items themselves.
For each raw item, formulate a specific question to verify its compliance with the requirements outlined in the document. The question should be in the format: "Is [item information] compliant with the requirements?"
When generating each question, include all relevant information about the item so that the question can be answered without needing to refer back to the items table. This is important because the team verifying the items will not have access to the table.
Before providing the final questions, write out your thought process and reasoning inside <scratchpad> tags. Explain how you extracted the raw items and formulated each question.
Finally, output the generated questions inside <questions> tags, with each question on a separate line.

Remember, the goal is to create specific, informative questions for each raw item to verify compliance with the requirements outlined in the document. Make sure to provide all necessary context within the questions themselves.
"""

In [99]:
prompt = PromptTemplate(
    template=generic_prompt,
    input_variables=["DOCUMENT","DOCUMENT_CONTEXT"],
)

In [47]:
# specific_context = """
# You will be generating questions to verify the compliance of ingredients based on a provided document and ingredients table. Here is the document:

# <document>
# {{DOCUMENT}}
# </document>

# And here is the ingredients table:

# <ingredients_table>
# {{INGREDIENTS_TABLE}}
# </ingredients_table>

# Please follow these steps to generate the questions:

# 1. Carefully review the document and ingredients table to understand the context and requirements.

# 2. Extract the individual raw ingredients from the ingredients table. For composed ingredients, break them down into their sub-ingredients. Focus on the raw ingredients only, not the composed ingredients themselves.

# 3. For each raw ingredient, formulate a specific question to verify its compliance with the company policy. The question should be in the format: "Is [ingredient information] compliant with the company policy?"

# 4. When generating each question, include all relevant information about the ingredient so that the question can be answered without needing to refer back to the ingredients table. This is important because the team verifying the ingredients will not have access to the table.

# 5. Before providing the final questions, write out your thought process and reasoning inside <scratchpad> tags. Explain how you extracted the raw ingredients and formulated each question.

# 6. Finally, output the generated questions inside <questions> tags, with each question on a separate line.

# Remember, the goal is to create specific, informative questions for each raw ingredient to verify compliance with the company policy. Make sure to provide all necessary context within the questions themselves.

# Please generate the questions now, following the provided instructions carefully."""

In [110]:
specific_context = """
The document is a list of specificity for a pastry product, every ingredient listed must be verified to be accepted by the Coup de Pates company.
Extract the individual raw ingredients from the ingredients table. For composed ingredients, break them down into their sub-ingredients. Focus on the raw ingredients only, not the composed ingredients themselves.
When generating each question, include all relevant information about the ingredient so that the question can be answered without needing to refer back to the ingredients table. This is important because the team verifying the ingredients will not have access to the table.
Look for the most detail in each ingredient, such as the labels (RSPCO, élevés en cage), do not add the country it is from.
Do not ask for an ingredient that is not raw, only raw ingredients are to be verified such as sugar, additifs, oil, colorants, etc.

The questions must be specific to a unique ingredient at each time.
The question will be asked directly to the "Charte Qualité" team, they won't have access to the provided document.
"""

In [111]:
llm = ChatOpenAI(model="gpt-4o", temperature = 0.1)
llm_chain = prompt | llm 

questions = llm_chain.invoke({"DOCUMENT": target_text, "DOCUMENT_CONTEXT": specific_context })

In [112]:
question_list = str(questions.content).split("\n")

In [113]:
for question in question_list:
    print(question)

<scratchpad>
1. **Review the document and items table**: The document provides a detailed list of ingredients for a pastry product, broken down into semi-finished products and their respective raw materials. Each raw material is listed with its percentage composition and additional details such as sub-ingredients for composed materials.

2. **Extract individual raw items**: 
   - For each semi-finished product, identify the raw materials.
   - For composed materials, break them down into their sub-ingredients.

3. **Formulate specific questions**: 
   - Each question should be specific to a unique raw ingredient.
   - Include all relevant information about the ingredient in the question.
   - Ensure the question is in French, as the document is in French.

4. **Example breakdown**:
   - For "1150038 LAMEQUICK CE RSPO", the sub-ingredients are: sirop de glucose, huile de palmiste, émulsifiant : E472a (acetic acid esters of mono- and diglycerides of fatty acids), protéine de lait.
   - F

In [116]:
from bs4 import BeautifulSoup

# Parse the text with BeautifulSoup
soup = BeautifulSoup(str(questions.content), 'html.parser')

# Find the content inside <questions>
questions_content = soup.find('questions').text

print(questions_content)


Is "framboise" compliant with the requirements?
Is "eau" compliant with the requirements?
Is "sucre cristal" compliant with the requirements?
Is "sirop de glucose" (from LAMEQUICK CE RSPO) compliant with the requirements?
Is "huile de palmiste" (from LAMEQUICK CE RSPO) compliant with the requirements?
Is "émulsifiant : E472a (acetic acid esters of mono- and diglycerides of fatty acids)" (from LAMEQUICK CE RSPO) compliant with the requirements?
Is "protéine de lait" (from LAMEQUICK CE RSPO) compliant with the requirements?
Is "crème de lait" (from CREME 35% UHT 10L DDJ) compliant with the requirements?
Is "stabilisant : E407" (from CREME 35% UHT 10L DDJ) compliant with the requirements?
Is "gélatine bovine" compliant with the requirements?
Is "œuf entier" compliant with the requirements?
Is "pistache" (from PUR PATE DE PISTACHE) compliant with the requirements?
Is "lait entier" compliant with the requirements?
Is "arôme pistache naturel" compliant with the requirements?
Is "spiruline v

In [122]:
llm = ChatOpenAI(model="gpt-4o", temperature = 0.1)

prompt = PromptTemplate(
    template="""Traduit ces questions en francais : {questions} 
    
    Renvois un une liste avec les questions directement : """,
    input_variables=["questions"],
)

llm_chain = prompt | llm 

fr_questions = llm_chain.invoke({"questions": questions_content })

In [125]:
print(fr_questions.content)

- Est-ce que "framboise" est conforme aux exigences ?
- Est-ce que "eau" est conforme aux exigences ?
- Est-ce que "sucre cristal" est conforme aux exigences ?
- Est-ce que "sirop de glucose" (de LAMEQUICK CE RSPO) est conforme aux exigences ?
- Est-ce que "huile de palmiste" (de LAMEQUICK CE RSPO) est conforme aux exigences ?
- Est-ce que "émulsifiant : E472a (esters acétiques de mono- et diglycérides d'acides gras)" (de LAMEQUICK CE RSPO) est conforme aux exigences ?
- Est-ce que "protéine de lait" (de LAMEQUICK CE RSPO) est conforme aux exigences ?
- Est-ce que "crème de lait" (de CREME 35% UHT 10L DDJ) est conforme aux exigences ?
- Est-ce que "stabilisant : E407" (de CREME 35% UHT 10L DDJ) est conforme aux exigences ?
- Est-ce que "gélatine bovine" est conforme aux exigences ?
- Est-ce que "œuf entier" est conforme aux exigences ?
- Est-ce que "pistache" (de PUR PATE DE PISTACHE) est conforme aux exigences ?
- Est-ce que "lait entier" est conforme aux exigences ?
- Est-ce que "arô